In [1]:
# !tar -xvf hackathon_files_for_participants_ozon.tar.gz

In [2]:
# !pip install catboost
# !pip install pandas
# !pip install -U scikit-learn
# !pip install seaborn
# !pip install pyarrow
# !pip install transformers
# !pip install setfit
# !pip install torch
# !pip install -U protobuf==3.20
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

In [3]:
# COMMENT: Just curios, why this logger? 
import datetime
import logging
import sys
from threading import Thread
# Cloudwatch logging
logger = logging.getLogger()

def setup_file_logger():
    log_file = "{}-{}.log".format(MODEL_CKPT.replace("/", "_"), str(datetime.datetime.now()))
    hdlr = logging.FileHandler(log_file)
    formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
    hdlr.setFormatter(formatter)
    logger.addHandler(hdlr) 
    logger.setLevel(logging.INFO)

def log(message, type='info'):
    #outputs to Jupyter console
    print('{} {}'.format(datetime.datetime.now(), message))
    #outputs to file
    if type == 'info':
        logger.info(message)
    elif type == 'warning':
        logger.warning(message)
    elif type == 'error':
        logger.error(message)
    elif type == 'critical':
        logger.critical(message)

threaded_logging = Thread(target=setup_file_logger)
threaded_logging.start()
threaded_logging.join()
print("Background logger started")

# Cloudwatch logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# stdout logging
stream_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(stream_handler)

Exception in thread Thread-5 (setup_file_logger):
Traceback (most recent call last):
  File "/root/miniconda3/envs/python3.10/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/root/miniconda3/envs/python3.10/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_1801/3159764320.py", line 10, in setup_file_logger
NameError: name 'MODEL_CKPT' is not defined


Background logger started


In [4]:
!nvidia-smi

Mon May 29 12:43:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
|  0%   53C    P5    38W / 170W |      1MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import json
from functools import partial
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from scipy.spatial.distance import cosine, euclidean
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split

import torch
import transformers
from datasets import Dataset, load_dataset
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          EarlyStoppingCallback, IntervalStrategy, Trainer,
                          TrainingArguments)
from transformers import logging as transformers_logging

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score

In [6]:
def process_df(features):
    features = features[["name1", "name2", "target"]]
    features["text"] = features["name1"] + "[SEP]" + features["name2"]
    features["label"] = features["target"].astype(int)
    features = features[["text", "label"]]
    features = features.drop_duplicates()
    features = features.reset_index(drop=True)
    return features

In [7]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"], max_length=256, padding="longest", truncation=True
    )

def get_hf_dataset(train_df, val_df):
    hf_datasets = dict()
    for key, value in [("train", train_df), ("val", val_df)]:
        dataset = Dataset.from_pandas(value, split="key")
        hf_datasets[key] = dataset.map(tokenize_function, batched=True)
    return hf_datasets


def tokenize(message, device="cuda", **kwargs):
    text_enc = tokenizer.encode_plus(
        message,
        None,
        add_special_tokens=True,
        max_length=CONFIG["max_token_count"],
        padding="max_length",
        return_token_type_ids=False,
        return_attention_mask=True,
        truncation=True,
        return_tensors="np",
    )
    input_ids = torch.tensor(text_enc["input_ids"], dtype=torch.long).to(device)
    attention_mask = torch.tensor(text_enc["attention_mask"], dtype=torch.long).to(
        device
    )
    return input_ids, attention_mask

sigmoid = torch.nn.Sigmoid()
    
def inference(message, **kwargs):
    input_ids, attention_mask = tokenize(message, **kwargs)
    preds = model(input_ids, attention_mask)
    logits = sigmoid(preds["logits"])[0].detach().cpu().numpy()
    return logits


In [8]:
MODEL_CKPT = "microsoft/deberta-v3-base"
MODEL_PATH = "/predefined_classes_best"

CONFIG = {"max_token_count": 512}
BATCH_SIZE = 32

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_CKPT, use_fast=True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/miniconda3/envs/python3.10/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
def compute_metrics(eval_pred):
    """A function for computing metrics for multi-label HF-classifier models

    Args:
        eval_pred (_type_): _description_

    Returns:
        _type_: _description_
    """
    predictions, labels = eval_pred
    pred_labels = np.argmax(predictions, axis=1)

    f1_results = f1_score(labels, pred_labels)
    precision_results = precision_score(labels, pred_labels)
    recall_results = recall_score(labels, pred_labels)

    output = {"f1": f1_results, "precision": precision_results, "recall": recall_results}
    logger.info(output)
    return output

In [11]:
# import gc
# import torch

# for fold in range(5):
#     train = pd.read_parquet(f"/root/fold_{fold}_train.parquet")
#     test = pd.read_parquet(f"/root/fold_{fold}_test.parquet")
#     val = pd.read_parquet(f"/root/fold_{fold}_val.parquet")
    
#     train = process_df(train)
#     test = process_df(test)
#     val = process_df(val)
        
#     train_texts = set(train["text"].values)
#     val = val[val["text"].apply(lambda x: x not in train_texts)]
#     test = test[test["text"].apply(lambda x: x not in train_texts)]

#     best_model_name = MODEL_PATH
#     hf_datasets = get_hf_dataset(train, val)
#     class_names = [0, 1]
#     id2label = {i: label for i, label in enumerate(class_names)}
#     if "mapping" not in CONFIG:
#         CONFIG['mapping'] = {'id_to_anecdote': id2label}
#     # init model
#     model = AutoModelForSequenceClassification.from_pretrained(
#         MODEL_CKPT,
#         num_labels=2,
#         problem_type="single_label_classification",
#         ).to('cuda')
#     model.config.id2label = id2label

#     args = TrainingArguments(
#         output_dir="predefined_model_new",
#         evaluation_strategy = "epoch",
#         save_strategy = "epoch",
#         learning_rate=1e-4,
#         per_device_train_batch_size=BATCH_SIZE,
#         per_device_eval_batch_size=BATCH_SIZE,
#         #num_train_epochs=10000,
#         num_train_epochs=1,
#         weight_decay=0.01,
#         save_total_limit=1,
#         metric_for_best_model = 'f1',
#         load_best_model_at_end=True,
#         fp16=True,
#     #         report_to="tensorboard"
#     )

#     trainer = Trainer(
#         model,
#         args,
#         train_dataset=hf_datasets["train"],
#         eval_dataset=hf_datasets["val"],
#         compute_metrics=compute_metrics,
#         tokenizer=tokenizer,
#         callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
#     )

#     prev_score = -1
#     best_model_checkpoint = None

#     trainer.evaluate()
#     trainer.train()
#     results = trainer.evaluate()

#     trainer.save_model(f"{best_model_name}_{fold}")
    
#     del trainer
#     del model
#     torch.cuda.empty_cache()
#     gc.collect()


In [12]:
# import gc
# import torch

# for fold in range(5):
#     test = pd.read_parquet(f"/root/fold_{fold}_test.parquet")
#     test = process_df(test)
#     test = test[test["text"].apply(lambda x: x not in train_texts)]

#     best_model_name = MODEL_PATH
#     hf_datasets = get_hf_dataset(test[:10], test)
#     class_names = [0, 1]
#     id2label = {i: label for i, label in enumerate(class_names)}
#     if "mapping" not in CONFIG:
#         CONFIG['mapping'] = {'id_to_anecdote': id2label}
#     # init model
#     model = AutoModelForSequenceClassification.from_pretrained(
#         f"/predefined_classes_best_{fold}",
#         num_labels=2,
#         problem_type="single_label_classification",
#         ).to('cuda')
#     model.config.id2label = id2label

#     args = TrainingArguments(
#         output_dir="predefined_model_new",
#         evaluation_strategy = "epoch",
#         save_strategy = "epoch",
#         learning_rate=1e-4,
#         per_device_train_batch_size=BATCH_SIZE,
#         per_device_eval_batch_size=BATCH_SIZE,
#         #num_train_epochs=10000,
#         num_train_epochs=1,
#         weight_decay=0.01,
#         save_total_limit=1,
#         metric_for_best_model = 'f1',
#         load_best_model_at_end=True,
#         fp16=True,
#     #         report_to="tensorboard"
#     )

#     trainer = Trainer(
#         model,
#         args,
#         train_dataset=hf_datasets["train"],
#         eval_dataset=hf_datasets["val"],
#         compute_metrics=compute_metrics,
#         tokenizer=tokenizer,
#         callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
#     )

#     prev_score = -1
#     best_model_checkpoint = None

#     trainer.evaluate()
    
#     del trainer
#     del model
#     torch.cuda.empty_cache()
#     gc.collect()


In [13]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve, auc


def pr_auc_macro(
    df: pd.DataFrame,
    prec_level: float = 0.75,
    cat_column: str = "cat3_grouped"
) -> float:

    y_true = df["label"]
    y_pred = df["scores"]
    categories = df[cat_column]

    weights = []
    pr_aucs = []

    unique_cats, counts = np.unique(categories, return_counts=True)

    # calculate metric for each big category
    for i, category in enumerate(unique_cats):
        # take just a certain category
        cat_idx = np.where(categories == category)[0]
        y_pred_cat = y_pred[cat_idx]
        y_true_cat = y_true[cat_idx]

        # if there is no matches in the category then PRAUC=0
        if sum(y_true_cat) == 0:
            pr_aucs.append(0)
            weights.append(counts[i] / len(categories))
            continue
        
        # get coordinates (x, y) for (recall, precision) of PR-curve
        y, x, _ = precision_recall_curve(y_true_cat, y_pred_cat)
        
        # reverse the lists so that x's are in ascending order (left to right)
        y = y[::-1]
        x = x[::-1]
        
        # get indices for x-coordinate (recall) where y-coordinate (precision) 
        # is higher than precision level (75% for our task)
        good_idx = np.where(y >= prec_level)[0]
        
        # if there are more than one such x's (at least one is always there, 
        # it's x=0 (recall=0)) we get a grid from x=0, to the rightest x 
        # with acceptable precision
        if len(good_idx) > 1:
            gt_prec_level_idx = np.arange(0, good_idx[-1] + 1)
        # if there is only one such x, then we have zeros in the top scores 
        # and the curve simply goes down sharply at x=0 and does not rise 
        # above the required precision: PRAUC=0
        else:
            pr_aucs.append(0)
            weights.append(counts[i] / len(categories))
            continue
        
        # calculate category weight anyway
        weights.append(counts[i] / len(categories))
        # calculate PRAUC for all points where the rightest x 
        # still has required precision 
        try:
            pr_auc_prec_level = auc(x[gt_prec_level_idx], y[gt_prec_level_idx])
            if not np.isnan(pr_auc_prec_level):
                pr_aucs.append(pr_auc_prec_level)
        except ValueError:
            pr_aucs.append(0)
            
    return np.average(pr_aucs, weights=weights)

In [14]:
import gc
import torch
from transformers import pipeline

for fold in range(2, 5):

    features = pd.read_parquet(f"/root/fold_{fold}_test.parquet")

    features["text"] = features["name1"] + "[SEP]" + features["name2"]
    features["label"] = features["target"].astype(int)
    features = features.loc[features[["text", "label"]].drop_duplicates().index]

    features = features.reset_index(drop=True)
    test = features
    
    pipe = pipeline("text-classification", f"/predefined_classes_best_{fold}", device=0, batch_size=32)
    preds = pipe(list(test["text"].values))
    preds = [p["score"] if p["label"] else 1 - p["score"] for p in preds]
    test["scores"] = preds
    print(fold, pr_auc_macro(test))
    del pipe
    torch.cuda.empty_cache()
    gc.collect()

2 0.6737361076111674
3 0.08432013986586123
4 0.675490557871805
